## Importing neccessary libraries

In [29]:
import os
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from ultralytics import YOLO


## Handling the raw data

In [2]:
# Define paths
base_path = '/Users/macbookpro/Desktop/thesis_project/PeopleArt'
jpeg_images_path = os.path.join(base_path, 'JPEGImages')
annotations_path = os.path.join(base_path, 'Annotations')

In [3]:
image_folders = os.listdir(jpeg_images_path)
image_folders = [folder for folder in image_folders if folder != ".DS_Store"]
image_folders.sort()
print("Folders of classes:", image_folders)

Folders of classes: ['Academicism', 'AnalyticalRealism', 'ArtDeco', 'ArtNouveau(Modern)', 'Biedermeier', 'Classicism', 'Constructivism', 'Cubism', 'Cubo-Futurism', 'Divisionism', 'EnvironmentalArt', 'FantasticRealism', 'FeministArt', 'HighRenaissance', 'Impressionism', 'InternationalGothic', 'Japonism', 'LowbrowArt', 'MagicRealism', 'MechanisticCubism', 'Muralism', 'Naturalism', 'Neo-FigurativeArt', 'Neo-Rococo', 'Neo-baroque', 'NewEuropeanPainting', 'NorthernRenaissance', 'Outsiderart', 'Photorealism', 'PopArt', 'PosterArtRealism', 'ProtoRenaissance', 'Realism', 'Rococo', 'Shin-hanga', 'SocialistRealism', 'Suprematism', 'Symbolism', 'Synthetism', 'Tonalism', 'Ukiyo-e', 'cartoon', 'photo']


In [15]:
test_path = '/Users/macbookpro/Desktop/thesis_project/PeopleArt/data_w_styles/images/test'
folders = os.listdir(test_path)
folders = [folder for folder in folders if folder != ".DS_Store"]
folders.sort()
print(folders)


['Academicism', 'AnalyticalRealism', 'ArtDeco', 'ArtNouveau(Modern)', 'Biedermeier', 'Classicism', 'Constructivism', 'Cubism', 'Cubo-Futurism', 'Divisionism', 'EnvironmentalArt', 'FantasticRealism', 'FeministArt', 'HighRenaissance', 'Impressionism', 'InternationalGothic', 'Japonism', 'LowbrowArt', 'MagicRealism', 'MechanisticCubism', 'Muralism', 'Naturalism', 'Neo-FigurativeArt', 'Neo-Rococo', 'Neo-baroque', 'NewEuropeanPainting', 'NorthernRenaissance', 'Photorealism', 'PopArt', 'PosterArtRealism', 'ProtoRenaissance', 'Realism', 'Rococo', 'Shin-hanga', 'SocialistRealism', 'Suprematism', 'Symbolism', 'Synthetism', 'Tonalism', 'Ukiyo-e', 'cartoon', 'photo']


In [17]:
images_files = []

for folder in folders:
    folder_path = os.path.join(test_path, folder)
    for image_file in os.listdir(folder_path):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Assicurati che sia un'immagine
            image_path = os.path.join(folder_path, image_file)
            images_files.append((image_path))


In [19]:
len(images_files)

132

In [5]:
anno_files = []

for folder in image_folders:
    folder_path = os.path.join(annotations_path, folder)
    for image_file in os.listdir(folder_path):
        if image_file.lower().endswith(('xml')):  # Assicurati che sia un'immagine
            image_path = os.path.join(folder_path, image_file)
            anno_files.append((image_path))



In [6]:
anno_files.sort()
images_files.sort()

In [7]:
len(images_files)

4778

In [8]:
len(anno_files)

1644

In [9]:
os.path.splitext(os.path.basename(anno_files[0]))[0]

'albert-anker_b-ckligumpen-1866.jpg'

In [10]:
os.path.basename(images_files[0])

'albert-anker_b-ckligumpen-1866.jpg'

In [11]:
matched_images = []
matched_annotations = []
for image in images_files:
    for file in anno_files:
        if os.path.basename(image) == os.path.splitext(os.path.basename(file))[0]:
            matched_images.append(image)
            matched_annotations.append(file)

## splitting into training and testing data

In [12]:
train_images, val_images, train_annotations, val_annotations = train_test_split(matched_images, matched_annotations, test_size=0.2, random_state=42)


In [13]:
train_images, val_images, train_annotations, val_annotations = train_test_split(train_images, train_annotations, test_size=0.1, random_state=42)


In [14]:
os.makedirs(os.path.join(base_path, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(base_path, 'images/test'), exist_ok=True)
os.makedirs(os.path.join(base_path, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(base_path, 'labels/test'), exist_ok=True)

In [15]:
def copy_files(img_files, ann_files, img_dst_dir, ann_dst_dir):
    for img_file, ann_file in zip(img_files, ann_files):
        img_class = os.path.basename(os.path.dirname(img_file))
        img_dst_class_dir = os.path.join(img_dst_dir, img_class)
        ann_dst_class_dir = os.path.join(ann_dst_dir, img_class)
        
        os.makedirs(img_dst_class_dir, exist_ok=True)
        os.makedirs(ann_dst_class_dir, exist_ok=True)
        
        shutil.copy(img_file, os.path.join(img_dst_class_dir, os.path.basename(img_file)))
        shutil.copy(ann_file, os.path.join(ann_dst_class_dir, os.path.basename(ann_file)))



In [16]:
# Copy train and val files to the new directory structure
copy_files(train_images, train_annotations, os.path.join(base_path, 'images/train'), os.path.join(base_path, 'labels/train'))
copy_files(val_images, val_annotations, os.path.join(base_path, 'images/test'), os.path.join(base_path, 'labels/val'))

print(f"Training set contains {len(train_images)} images.")
print(f"Validation set contains {len(val_images)} images.")

Training set contains 1183 images.
Validation set contains 132 images.


## converting xml files into txt files for yolo format annotations

In [27]:
def convert_voc_to_yolo(xml_file):
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # Get image dimensions
        size_element = root.find('size')
        if size_element is None:
            print(f"Error: 'size' element not found in {xml_file}")
            return []

        width_element = size_element.find('width')
        height_element = size_element.find('height')
        if width_element is None or height_element is None:
            print(f"Error: 'width' or 'height' element not found in {xml_file}")
            return []

        width = int(width_element.text)
        height = int(height_element.text)

        yolo_annotations = []

        # Find all objects
        for obj in root.findall('object'):
            class_name_element = obj.find('name')
            if class_name_element is None:
                print(f"Error: 'name' element not found in an object in {xml_file}")
                continue

            class_name = class_name_element.text
            if class_name != 'person':
                print(f"Unexpected class '{class_name}' found in {xml_file}. Skipping.")
                continue  # Skip objects that are not 'person'

            class_id = 0  # Since 'person' is the only class, its ID is 0

            bndbox = obj.find('bndbox')
            if bndbox is None:
                print(f"Error: 'bndbox' element not found in an object in {xml_file}")
                continue

            xmin_element = bndbox.find('xmin')
            ymin_element = bndbox.find('ymin')
            xmax_element = bndbox.find('xmax')
            ymax_element = bndbox.find('ymax')

            if xmin_element is None or ymin_element is None or xmax_element is None or ymax_element is None:
                print(f"Error: One of 'xmin', 'ymin', 'xmax', 'ymax' elements not found in {xml_file}")
                continue

            xmin = float(xmin_element.text)
            ymin = float(ymin_element.text)
            xmax = float(xmax_element.text)
            ymax = float(ymax_element.text)

            x_center = (xmin + xmax) / 2.0 / width
            y_center = (ymin + ymax) / 2.0 / height
            box_width = (xmax - xmin) / width
            box_height = (ymax - ymin) / height

            yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")

        return yolo_annotations
    except Exception as e:
        print(f"Error processing file {xml_file}: {e}")
        return []

def process_folder(folder, output_folder):
    # Create the new output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Walk through the directory structure
    for root, _, files in os.walk(folder):
        # Determine the corresponding output subdirectory
        relative_path = os.path.relpath(root, folder)
        output_subdir = os.path.join(output_folder, relative_path)
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)

        for file in files:
            if file.endswith('.xml'):
                xml_file = os.path.join(root, file)
                yolo_annotations = convert_voc_to_yolo(xml_file)

                # Write YOLO annotations to the corresponding output file
                txt_filename_dummy = os.path.splitext(file)[0] 
                txt_filename=txt_filename_dummy[:-3] + 'txt'
                output_file = os.path.join(output_subdir, txt_filename)
                with open(output_file, 'w') as f:
                    for annotation in yolo_annotations:
                        f.write(f"{annotation}\n")
                print(f"Processed {xml_file} -> {output_file}")



In [28]:
source_folder = '/Users/macbookpro/Desktop/thesis_project/PeopleArt/Annotations'
output_folder = '/Users/macbookpro/Desktop/thesis_project/PeopleArt/txt_annotations'
process_folder(source_folder, output_folder)

Processed /Users/macbookpro/Desktop/thesis_project/PeopleArt/Annotations/Shin-hanga/asano-takeji_twilight-in-the-village-nara-1953-shin.jpg.xml -> /Users/macbookpro/Desktop/thesis_project/PeopleArt/txt_annotations/Shin-hanga/asano-takeji_twilight-in-the-village-nara-1953-shin.txt
Processed /Users/macbookpro/Desktop/thesis_project/PeopleArt/Annotations/Shin-hanga/natori-shunsen_lion-dance-1950.jpg.xml -> /Users/macbookpro/Desktop/thesis_project/PeopleArt/txt_annotations/Shin-hanga/natori-shunsen_lion-dance-1950.txt
Processed /Users/macbookpro/Desktop/thesis_project/PeopleArt/Annotations/Shin-hanga/hasui-kawase_snow-at-hinoeda-shrine-1931.jpg.xml -> /Users/macbookpro/Desktop/thesis_project/PeopleArt/txt_annotations/Shin-hanga/hasui-kawase_snow-at-hinoeda-shrine-1931.txt
Processed /Users/macbookpro/Desktop/thesis_project/PeopleArt/Annotations/Shin-hanga/kiyokata-kaburagi_woman-and-white-swans-1906.jpg.xml -> /Users/macbookpro/Desktop/thesis_project/PeopleArt/txt_annotations/Shin-hanga/kiy

In [ ]:
source_folder = '/Users/macbookpro/Desktop/thesis_project/PeopleArt/labels_xml/val'
output_folder = '/Users/macbookpro/Desktop/thesis_project/PeopleArt/labels/val'
process_folder(source_folder, output_folder)

Processed /Users/macbookpro/Desktop/thesis_project/PeopleArt/labels_xml/val/Shin-hanga/asano-takeji_twilight-in-the-village-nara-1953-shin.jpg.xml -> /Users/macbookpro/Desktop/thesis_project/PeopleArt/labels/val/Shin-hanga/asano-takeji_twilight-in-the-village-nara-1953-shin.txt
Processed /Users/macbookpro/Desktop/thesis_project/PeopleArt/labels_xml/val/Shin-hanga/natori-shunsen_onoe-kikugoro-as-omatsuri-sanshichi-1931.jpg.xml -> /Users/macbookpro/Desktop/thesis_project/PeopleArt/labels/val/Shin-hanga/natori-shunsen_onoe-kikugoro-as-omatsuri-sanshichi-1931.txt
Processed /Users/macbookpro/Desktop/thesis_project/PeopleArt/labels_xml/val/Shin-hanga/goyo-hashiguchi_woman-holding-lipstick-1920.jpg.xml -> /Users/macbookpro/Desktop/thesis_project/PeopleArt/labels/val/Shin-hanga/goyo-hashiguchi_woman-holding-lipstick-1920.txt
Processed /Users/macbookpro/Desktop/thesis_project/PeopleArt/labels_xml/val/Shin-hanga/natori-shunsen_the-actor-nakamura-t-z-v-in-the-role-of-susanoo-no-mikoto-1915.jpg.xm

## YOLO transfer in original data

In [ ]:
data_yaml_content = """
train: /Users/macbookpro/Desktop/thesis_project/PeopleArt/data/images/train
val: /Users/macbookpro/Desktop/thesis_project/PeopleArt/data/images/val

nc: 1
names: ['person']
"""

# Define the path where to save the data.yaml file
data_yaml_path = '/Users/macbookpro/Desktop/thesis_project/PeopleArt/data.yaml'

# Save the data.yaml file
with open(data_yaml_path, 'w') as file:
    file.write(data_yaml_content)

print(f"data.yaml file saved to {data_yaml_path}")


data.yaml file saved to /Users/macbookpro/Desktop/thesis_project/PeopleArt/data.yaml


In [ ]:
import os

def verify_labels(image_dir, label_dir):
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    missing_labels = []
    empty_labels = []

    for image_file in image_files:
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(label_dir, label_file)
        
        if not os.path.exists(label_path):
            missing_labels.append(label_file)
        else:
            with open(label_path, 'r') as f:
                content = f.read().strip()
                if not content:
                    empty_labels.append(label_file)

    if missing_labels:
        print(f"Missing label files for the following images: {missing_labels}")
    else:
        print("All images have corresponding label files.")

    if empty_labels:
        print(f"Empty label files for the following images: {empty_labels}")
    else:
        print("All label files are non-empty.")

# Define paths
val_image_dir = '/Users/macbookpro/Desktop/thesis_project/PeopleArt/data/images/val'
val_label_dir = '/Users/macbookpro/Desktop/thesis_project/PeopleArt/data/labels/val'

# Verify label files for validation set
verify_labels(val_image_dir, val_label_dir)


All images have corresponding label files.
All label files are non-empty.


In [ ]:
# Import the YOLO class
from ultralytics import YOLO

# Initialize the YOLOv8 model
model = YOLO('yolov8n.pt')  # yolov8n.pt is the pre-trained model weight file

# Train the model
model.train(data='/Users/macbookpro/Desktop/thesis_project/PeopleArt/data.yaml', epochs=10, batch=16, imgsz=640)


New https://pypi.org/project/ultralytics/8.2.28 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.27 🚀 Python-3.8.19 torch-2.3.0 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/macbookpro/Desktop/thesis_project/PeopleArt/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show

train: Scanning /Users/macbookpro/Desktop/thesis_project/PeopleArt/data/labels/train/Academicism... 1315 images, 119 backgrounds, 0 corrupt: 100%|██████████| 1315/1315 [00:00<00:00, 3521.40it/s]

train: New cache created: /Users/macbookpro/Desktop/thesis_project/PeopleArt/data/labels/train/Academicism.cache



val: Scanning /Users/macbookpro/Desktop/thesis_project/PeopleArt/data/labels/val/Academicism... 329 images, 35 backgrounds, 0 corrupt: 100%|██████████| 329/329 [00:00<00:00, 4644.98it/s]


val: New cache created: /Users/macbookpro/Desktop/thesis_project/PeopleArt/data/labels/val/Academicism.cache
Plotting labels to runs/detect/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train5
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        329        781      0.634      0.453      0.549      0.299



10 epochs completed in 3.359 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.2MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics YOLOv8.2.27 🚀 Python-3.8.19 torch-2.3.0 CPU (Apple M1)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [01:23<00:00,  7.57s/it]


                   all        329        781      0.633      0.453       0.55      0.299
Speed: 1.4ms preprocess, 247.1ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x294958040>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

In [ ]:
results = model.val(data='/Users/macbookpro/Desktop/thesis_project/PeopleArt/data.yaml')

In [ ]:
print(results)

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x292c36370>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

These are the results on the test data: 
- Precision: 0.6012061500860477;
- Recall: 0.4839948783610755;
- mAP50: 0.5472705657998922
- mAP50-95: 0.2978717535324559
- fitness: 0.3228116347591995

Based on precision and recall, the model seems to be decent at identifying the presence of humans (detection), but it's struggling to accurately draw tight bounding boxes around them (localization).

- Precision: The model is not misidentifying other objects as humans, hence, there is a low false positive rate which resulted in higher precision. 
- Lower Recall and mAP: The model is missing some humans or not drawing the bounding boxes accurately enough to meet the stricter IoU thresholds in mAP calculation.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import random
from pathlib import Path
import yaml


In [ ]:
# Initialize the YOLO model with the best weights
model = YOLO('/Users/macbookpro/Desktop/thesis_project/runs/detect/train/weights/best.pt')  # Adjust the path to your best.pt file
results = model.val(data='/Users/macbookpro/Desktop/thesis_project/PeopleArt/data.yaml')


Ultralytics YOLOv8.2.15 🚀 Python-3.11.9 torch-2.3.0 CPU (Apple M1)
Model summary (fused): 168 layers, 3014033 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /Users/macbookpro/Desktop/thesis_project/PeopleArt/data/labels/val/Academicism.cache... 329 images, 35 backgrounds, 0 corrupt: 100%|██████████| 329/329 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:51<00:00,  2.46s/it]

                   all        329        781          0          0          0          0


Speed: 0.9ms preprocess, 150.2ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/val


## Setting up YOLO for augmented data

In [ ]:
augmented_data_yaml_content = """
train: /Users/macbookpro/Desktop/thesis_project/PeopleArt/augmented_data/images/train
val: /Users/macbookpro/Desktop/thesis_project/PeopleArt/augmented_data/images/val

nc: 1
names: ['person']
"""

# Define the path where to save the data.yaml file
augmented_data_yaml_path = '/Users/macbookpro/Desktop/thesis_project/PeopleArt/augmented_data.yaml'

# Save the data.yaml file
with open(augmented_data_yaml_path, 'w') as file:
    file.write(augmented_data_yaml_content)

print(f"data.yaml file saved to {augmented_data_yaml_path}")

data.yaml file saved to /Users/macbookpro/Desktop/thesis_project/PeopleArt/augmented_data.yaml


In [ ]:
# Import the YOLO class
from ultralytics import YOLO

# Initialize the YOLOv8 model
model = YOLO('yolov8n.pt')  # yolov8n.pt is the pre-trained model weight file

# Train the model
model.train(data='/Users/macbookpro/Desktop/thesis_project/PeopleArt/augmented_data.yaml', epochs=5, batch=16, imgsz=640)


New https://pypi.org/project/ultralytics/8.2.34 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.15 🚀 Python-3.11.9 torch-2.3.0 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/macbookpro/Desktop/thesis_project/PeopleArt/augmented_data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=

train: Scanning /Users/macbookpro/Desktop/thesis_project/PeopleArt/augmented_data/labels/train.cache... 6575 images, 595 backgrounds, 0 corrupt: 100%|██████████| 6575/6575 [00:00<?, ?it/s]

train: WARNING ⚠️ /Users/macbookpro/Desktop/thesis_project/PeopleArt/augmented_data/images/train/janos-mattis-teutsch_composition-1927_4.jpg: 1 duplicate labels removed


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/Users/macbookpro/anaconda3/envs/envallen/lib/python3.11/site-packages/albumentations/core/composition.py:144: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
val: Scanning /Users/macbookpro/Desktop/thesis_project/PeopleArt/augmented_data/labels/val.cache... 329 images, 35 backgrounds, 0 corrupt: 100%|██████████| 329/329 [00:00<?, ?it/s]


...

       7/10         0G      1.627      1.954      1.982         35        640: 100%|██████████| 411/411 [58:39<00:00,  8.56s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [01:24<00:00,  7.70s/it]

                   all        329        781      0.496       0.37       0.41      0.199



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/411 [00:00<?, ?it/s]

: 

## YOLO Predictions on 43 different styles

In [58]:
import os
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('runs/detect/train3/weights/best.pt')

# Specify the folder containing the images
folder_path = '/Users/macbookpro/Desktop/thesis_project/PeopleArt/data_w_styles/images/val/photo'

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    # Construct the full image path
    image_path = os.path.join(folder_path, filename)
    
    # Check if the file is an image (you can add more image extensions if needed)
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        # Predict using the YOLO model
        model.predict(image_path, save=True, imgsz=320, conf=0.5)



image 1/1 /Users/macbookpro/Desktop/thesis_project/PeopleArt/data_w_styles/images/val/photo/008122.jpg: 224x320 2 persons, 20.8ms
Speed: 0.6ms preprocess, 20.8ms inference, 0.3ms postprocess per image at shape (1, 3, 224, 320)
Results saved to runs/detect/predict2

image 1/1 /Users/macbookpro/Desktop/thesis_project/PeopleArt/data_w_styles/images/val/photo/003088.jpg: 256x320 1 person, 26.3ms
Speed: 0.8ms preprocess, 26.3ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 320)
Results saved to runs/detect/predict2

image 1/1 /Users/macbookpro/Desktop/thesis_project/PeopleArt/data_w_styles/images/val/photo/006492.jpg: 256x320 (no detections), 25.8ms
Speed: 0.8ms preprocess, 25.8ms inference, 0.2ms postprocess per image at shape (1, 3, 256, 320)
Results saved to runs/detect/predict2

image 1/1 /Users/macbookpro/Desktop/thesis_project/PeopleArt/data_w_styles/images/val/photo/005873.jpg: 320x224 1 person, 26.1ms
Speed: 0.5ms preprocess, 26.1ms inference, 0.3ms postprocess per im

# testing the model with and without training 

In [11]:
source = '/Users/macbookpro/Desktop/thesis_project/PeopleArt/data_w_styles/images/test/Academicism/frederic-leighton_phoebe.jpg'

## with training

In [21]:
from PIL import Image

from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO("/Users/macbookpro/Desktop/thesis_project/runs/detect/train3/weights/best.pt")

# Run inference on 'bus.jpg'
results = model(images_files)  # results list

# Visualize the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    #r.show()

    # Save results to disk
    r.save(filename=f"results{i}.jpg")


0: 640x640 11 persons, 108.3ms
1: 640x640 9 persons, 108.3ms
2: 640x640 1 person, 108.3ms
3: 640x640 2 persons, 108.3ms
4: 640x640 1 person, 108.3ms
5: 640x640 1 person, 108.3ms
6: 640x640 (no detections), 108.3ms
7: 640x640 1 person, 108.3ms
8: 640x640 1 person, 108.3ms
9: 640x640 3 persons, 108.3ms
10: 640x640 3 persons, 108.3ms
11: 640x640 2 persons, 108.3ms
12: 640x640 6 persons, 108.3ms
13: 640x640 6 persons, 108.3ms
14: 640x640 7 persons, 108.3ms
15: 640x640 5 persons, 108.3ms
16: 640x640 8 persons, 108.3ms
17: 640x640 1 person, 108.3ms
18: 640x640 15 persons, 108.3ms
19: 640x640 4 persons, 108.3ms
20: 640x640 6 persons, 108.3ms
21: 640x640 3 persons, 108.3ms
22: 640x640 1 person, 108.3ms
23: 640x640 2 persons, 108.3ms
24: 640x640 3 persons, 108.3ms
25: 640x640 2 persons, 108.3ms
26: 640x640 1 person, 108.3ms
27: 640x640 3 persons, 108.3ms
28: 640x640 3 persons, 108.3ms
29: 640x640 4 persons, 108.3ms
30: 640x640 (no detections), 108.3ms
31: 640x640 2 persons, 108.3ms
32: 640x640

## without training

In [22]:
from PIL import Image

from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO("yolov8n.pt")

# Run inference on 'bus.jpg'
results = model(images_files)  # results list

# Visualize the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    #r.show()

    # Save results to disk
    #r.save(filename=f"results{i}.jpg")


0: 640x640 7 persons, 116.5ms
1: 640x640 8 persons, 1 horse, 116.5ms
2: 640x640 1 person, 116.5ms
3: 640x640 1 person, 116.5ms
4: 640x640 (no detections), 116.5ms
5: 640x640 1 bird, 1 tv, 116.5ms
6: 640x640 2 chairs, 116.5ms
7: 640x640 (no detections), 116.5ms
8: 640x640 1 person, 116.5ms
9: 640x640 2 persons, 116.5ms
10: 640x640 1 person, 116.5ms
11: 640x640 (no detections), 116.5ms
12: 640x640 5 persons, 116.5ms
13: 640x640 3 persons, 116.5ms
14: 640x640 5 persons, 2 horses, 116.5ms
15: 640x640 6 persons, 116.5ms
16: 640x640 6 persons, 1 bird, 1 potted plant, 116.5ms
17: 640x640 1 person, 1 umbrella, 1 bowl, 1 book, 116.5ms
18: 640x640 9 persons, 4 horses, 116.5ms
19: 640x640 2 persons, 2 sheeps, 116.5ms
20: 640x640 1 person, 116.5ms
21: 640x640 (no detections), 116.5ms
22: 640x640 1 person, 116.5ms
23: 640x640 (no detections), 116.5ms
24: 640x640 1 bed, 116.5ms
25: 640x640 (no detections), 116.5ms
26: 640x640 1 umbrella, 116.5ms
27: 640x640 1 bed, 116.5ms
28: 640x640 (no detections

# Summary of what was done here: 
#### I used the prediction mode in order to check how yolov8 does on the testing data before i train it on my data. i also did the prediction mode on the model i trained on my data. as a result i received two sets of testing images with bounding boxes and with confidence intervals for classifications, one set for raw yolo and one for my trained one. 
#### need to figure out how to compare these sets of images  
#### in addition, the output of the code was number of person objects detected per testing image. i want to store these numbers in a list for comparison or figure out how to retrieve this information.
#### need to compare the results with ground truth

the annotations of picture 002870.jpg list 10 person objects while the prediction of yolov8 detects 13 and trained yolov8 detects only 8. 

i have come to conclusion that the data needs filtering.

eda must be performed